In [1]:
def bytes_to_hex_string(byte_data, addSpaces = True):
    hex_bytes = [f"{byte:02x}" for byte in bytes(byte_data)]
    if addSpaces:
        return ' '.join(hex_bytes, )
    else:
        return ''.join(hex_bytes, )

import importlib
import _util
importlib.reload(_util)

from _types import BootAttrs
bootattrs = BootAttrs(
        version=258,
        max_packet_length=256,
        device_id=13398,
        erase_size=2048,
        write_size=8,
        memory_range=(6144, 174080),
        has_checksum=True,
    )
total_bytes, chunks, debug_parsed_records = _util.chunked("../mcbootflash/tests/testcases/flash/test.hex", bootattrs)

print(len(debug_parsed_records))

for d in debug_parsed_records:
    print(f"Segment({d[0]}, {d[1]}, hexStringToBytes(\"{bytes_to_hex_string(d[2], addSpaces=False)}\"), {d[3]}),")

number of debug_parsed_records 128
first debug_parsed_records  (0, 16, bytearray(b'\xe0\x1a\x04\x00\x00\x00\x00\x00\x04\x1a\x00\x00\x08\x1a\x00\x00'), 1)
second debug_parsed_records  (16, 32, bytearray(b'\x0c\x1a\x00\x00\x10\x1a\x00\x00\x14\x1a\x00\x00\x18\x1a\x00\x00'), 1)
n-th debug_parsed_records  (13296, 13304, bytearray(b'\x00\x00\x06\x00\xff\xff7\x00'), 1)
128
Segment(0, 16, hexStringToBytes("e01a040000000000041a0000081a0000"), 1),
Segment(16, 32, hexStringToBytes("0c1a0000101a0000141a0000181a0000"), 1),
Segment(32, 48, hexStringToBytes("1c1a0000001a0000201a0000241a0000"), 1),
Segment(48, 64, hexStringToBytes("281a00002c1a0000301a0000341a0000"), 1),
Segment(64, 80, hexStringToBytes("381a00003c1a0000401a0000441a0000"), 1),
Segment(80, 96, hexStringToBytes("481a00004c1a0000501a0000541a0000"), 1),
Segment(96, 112, hexStringToBytes("581a0000001a00005c1a0000601a0000"), 1),
Segment(112, 128, hexStringToBytes("641a0000681a00006c1a0000001a0000"), 1),
Segment(128, 144, hexStringToBytes("0

In [80]:

import binascii

def crc_ihex(hexstr):
    """Calculate the CRC for given Intel HEX hexstring.
    """
    crc = sum(binascii.unhexlify(hexstr))
    # print("sum of crc : ", crc)
    crc &= 0xff
    # print("after &= : ", crc)
    crc = ((~crc + 1) & 0xff)
    # print("final crc : ", crc)
    return crc
#    std::vector<uint8_t> bytes{67, 56, 89, 45};



def unpack_ihex(record):
    """Unpack given Intel HEX record into variables.
    """
    # Minimum :SSAAAATTCC, where SS is size, AAAA is address, TT is
    # type and CC is crc.
    if len(record) < 11:
        raise Error(f"record '{record}' too short")

    if record[0] != ':':
        raise Error(f"record '{record}' not starting with a ':'")

    value = bytearray.fromhex(record[1:])
    size = value[0]

    if size != len(value) - 5:
        raise Error(f"record '{record}' has wrong size")

    address = int.from_bytes(value[1:3], byteorder='big')
    type_ = value[3]
    data = value[4:-1]
    actual_crc = value[-1]

    crcBytes = record[1:-2]
    expected_crc = crc_ihex(crcBytes)
    print("crcBytes")
    print(crcBytes)

    print("expected_crc ", expected_crc)

    if actual_crc != expected_crc:
        raise Error(
            f"expected crc '{expected_crc:02X}' in record {record}, but got "
            f"'{actual_crc:02X}'")

    return (type_, address, size, data)


(type_, address, size, data) = unpack_ihex(":10000000E01A040000000000041A0000081A0000B2")
print("type_", type_)
print("address", address)
print("size", size)
print("data", bytes_to_hex_string(data))

# 10000000e01a040000000000041a0000081a0000
# from C++ : 
# 10000000e01a040000000000041a0000081a0000


crcBytes
10000000E01A040000000000041A0000081A0000
expected_crc  178
type_ 0
address 0
size 16


NameError: name 'bytes_to_hex_string' is not defined